In [4]:
import pandas as pd
import sqlite3

pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth', 5000)

In [2]:
gamel = pd.read_csv('game_log.csv')
parkс = pd.read_csv('park_codes.csv')
personс = pd.read_csv('person_codes.csv')
teamc = pd.read_csv('team_codes.csv')
appeart = pd.read_csv('appearance_type.csv')

/dataquest/system/env/python3/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (12,13,14,15,19,20,81,82,83,84,85,86,87,88,93,94,95,96,97,98,99,100,105,106,108,109,111,112,114,115,117,118,120,121,123,124,126,127,129,130,132,133,135,136,138,139,141,142,144,145,147,148,150,151,153,154,156,157,160) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
def run_query(q):
    with sqlite3.connect('mlb.db') as conn:
        return pd.read_sql(q, conn)
    
def run_command(c):
    with sqlite3.connect('mlb.db') as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        conn.execute(c)
        
def show_tables():
    q = '''SELECT
                name,
                type
            FROM sqlite_master
            WHERE type IN ("table", "view")'''
    
    return run_query(q)

In [9]:
with sqlite3.connect('mlb.db') as conn:
    try:
        gamel.to_sql('game_log', conn, index=False)
        parkс.to_sql('park_codes', conn, index=False)
        personс.to_sql('person_codes', conn, index=False)
        teamс.to_sql('team_codes', conn, index=False)
    except ValueError as err:
        print('ValueError:', err)

ValueError: Table 'game_log' already exists.

In [15]:
q = '''
ALTER TABLE game_log
ADD COLUMN game_id TEXT;
'''

run_command(q)

q = '''
UPDATE game_log
SET game_id = h_name || date || number_of_game;
'''

run_command(q)

In [16]:
q = '''
CREATE TABLE IF NOT EXISTS person(
    person_id TEXT PRIMARY KEY,
    first_name TEXT,
    last_name TEXT
);
'''
run_command(q)

q = '''
CREATE TABLE IF NOT EXISTS park(
    park_id TEXT PRIMARY KEY,
    name TEXT,
    nickname TEXT,
    city TEXT,
    state TEXT,
    notes TEXT
);
'''
run_command(q)

q = '''
CREATE TABLE IF NOT EXISTS league(
    league_id TEXT PRIMARY KEY,
    name TEXT
);
'''
run_command(q)

# q = '''
# CREATE TABLE IF NOT EXISTS appearance_type(
#     appearance_type_id INTEGER PRIMARY KEY,
#     name TEXT,
#     category TEXT
# );
# '''
# run_command(q)

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info,game_id
0,18710504,0,Thu,CL1,None,1,FW1,None,1,0,2,54.0,D,None,None,None,FOR01,200.0,120.0,000000000,010010000,30.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,None,6.0,1.0,None,-1.0,None,4.0,1.0,1.0,1.0,0.0,0.0,27.0,9.0,0.0,3.0,0.0,0.0,31.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,None,0.0,0.0,None,-1.0,None,3.0,1.0,0.0,0.0,0.0,0.0,27.0,3.0,3.0,1.0,1.0,0.0,boakj901,John Boake,None,None,None,None,None,None,None,None,None,None,paboc101,Charlie Pabor,lennb101,Bill Lennon,mathb101,Bobby Mathews,prata101,Al Pratt,None,None,None,None,prata101,Al Pratt,mathb101,Bobby Mathews,whitd102,Deacon White,2.0,kimbg101,Gene Kimball,4.0,paboc101,Charlie Pabor,7.0,allia101,Art Allison,8.0,white104,Elmer White,9.0,prata101,Al Pratt,1.0,sutte101,Ezra Sutton,5.0,carlj102,Jim Carleton,3.0,bassj101,John Bass,6.0,selmf101,Frank Sellman,5.0,mathb101,Bobby Mathews,1.0,foraj101,Jim Foran,3.0,goldw101,Wally Goldsmith,6.0,lennb101,Bill Lennon,2.0,caret101,Tom Carey,4.0,mince101,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,None,Y,FW1187105040
1,18710505,0,Fri,BS1,None,1,WS3,None,1,20,18,54.0,D,None,None,None,WAS01,5000.0,145.0,107000435,640113030,41.0,13.0,1.0,2.0,0.0,13.0,0.0,0.0,0.0,18.0,None,5.0,3.0,None,-1.0,None,12.0,1.0,6.0,6.0,1.0,0.0,27.0,13.0,10.0,1.0,2.0,0.0,49.0,14.0,2.0,0.0,0.0,11.0,0.0,0.0,0.0,10.0,None,2.0,1.0,None,-1.0,None,14.0,1.0,7.0,7.0,0.0,0.0,27.0,20.0,10.0,2.0,3.0,0.0,dobsh901,Henry Dobson,None,None,None,None,None,None,None,None,None,None,wrigh101,Harry Wright,younn801,Nick Young,spala101,Al Spalding,braia102,Asa Brainard,None,None,None,None,spala101,Al

In [ ]:
q = '''
INSERT OR IGNORE INTO person 
    SELECT                                   
       id,                                                           
       first,
       last
    FROM person_codes
'''
run_command(q)   

result = run_query('SELECT * FROM person LIMIT 5')
result

In [ ]:
q = '''
INSERT OR IGNORE INTO park 
    SELECT                                   
       park_id,                                                           
       name,
       aka,
       city,
       state,
       notes
    FROM park_codes
'''
run_command(q)   

result = run_query('SELECT * FROM park LIMIT 5')
result

In [ ]:
q = '''
INSERT OR IGNORE INTO league VALUES
    ("NL", "National League"),
    ("AL", "American League"),
    ("AA", "Double-A Minor League"),
    ("UA", "Union Association League"),
    ("FL", "Florida State League"),
    ("PL", "Players' League")
'''
run_command(q)



result = run_query('SELECT * FROM league')
result
# teamc['league'].value_counts()

In [ ]:
run_command('DROP TABLE IF EXISTS appearance_type')

with sqlite3.connect('mlb.db') as conn:
    try:
        appeart.to_sql('appearance_type', conn, index=False, if_exists='append')
    except ValueError as err:
        print('ValueError:', err)
        
result = run_query('SELECT * FROM appearance_type')
result.head()

In [ ]:
q = '''
CREATE TABLE IF NOT EXISTS team (
    team_id TEXT PRIMARY KEY,
    league_id TEXT,
    city TEXT,
    nickname TEXT,
    franch_id TEXT,
    FOREIGN KEY (league_id) REFERENCES league(league_id)
);
'''
run_command(q)

q = '''
INSERT OR IGNORE INTO team 
    SELECT                                   
       team_id,                                                           
       league,
       city,
       nickname,
       franch_id
    FROM team_codes
'''
run_command(q)

result = run_query('SELECT * FROM team')
result.head()

In [ ]:
q = '''
CREATE TABLE IF NOT EXISTS game (
    game_id TEXT PRIMARY KEY,
    date TEXT,
    number_of_game INTEGER,
    park_id TEXT,
    length_outs INTEGER,
    day INTEGER,
    completion TEXT,
    forefeit TEXT,
    protest TEXT,
    FOREIGN KEY (league_id) REFERENCES league(league_id)
);
'''
run_command(q)

q = '''
INSERT OR IGNORE INTO team 
    SELECT                                   
       team_id,                                                           
       league,
       city,
       nickname,
       franch_id
    FROM team_codes
'''
run_command(q)

result = run_query('SELECT * FROM game')
result.head()

In [ ]:
# result = run_query('SELECT * FROM game_log LIMIT 5')
result = run_query('SELECT DISTINCT protest FROM game_log LIMIT 5')
result